In [1]:
from utils.all import *

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import f1_score,precision_score,recall_score
import itertools
import joblib
import os

In [3]:
sql="""
SELECT * FROM `uw-partner-prod.experimental.lead_conversions` 
WHERE prospect_creation_date >= '2020-10-01'

"""

In [4]:
raw_df=download_data(project_id='uw-data-warehouse-prod',query=sql)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=4khLkQwCexbcmn2qMFFz0eVHDKDwp9&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWjbX0Zl6O1CYRcA66gK8GNaAd7DUeWhS-TdN3CyAqr0u23nG-QAJho


Downloading:   0%|          | 0/155123 [00:00<?, ?rows/s]

In [7]:
raw_df['customer_ltv']=raw_df.customer_ltv.astype(float)

In [8]:
def split_train_valid(df, split_pct=.8):
    """Utility function to split the `df` into train and validation dataset, sorting by the `prospect_creation_date`"""
    validation_split_date = np.quantile(df.prospect_creation_date, q=split_pct)
    valid = df.loc[df.prospect_creation_date >= validation_split_date].reset_index(drop=True).copy()
    train = df.loc[df.prospect_creation_date < validation_split_date].reset_index(drop=True).copy()
    return train, valid

In [7]:
def compute_cost(df, failed_cost, value_col):
    value = df.loc[df['converted'],value_col].sum()
    price = (~df['converted']).sum() * failed_cost
    return price - value

def recommendation_regret(df, false_positive_cost, 
                          recommendation_col='recommended', 
                          value_col='customer_ltv'): ## Use "recommended" if you pick X=1
    recommended = df.loc[df[recommendation_col]].copy()# positives 
    not_recommended = df.loc[~df[recommendation_col]].copy() # negatives 
    recommended_cost =  compute_cost(recommended, false_positive_cost, value_col) # -(TP customer values - FP efforts) TP*x-Y*FP
    not_recommended_cost = - compute_cost(not_recommended, false_positive_cost, value_col )# TN cost saved -FN losing customer price 
    return recommended_cost + not_recommended_cost                     # value= (~df['converted']).sum() * failed_cost
                                                                       # price = FN *X
                                                                       # value-price= -(*price-*value)

The way we set $Y$ means that if we were to recommend all prospects or no prospect at all, the regret would be 0. That is, recommending everyone or nobody is the same as not having a recommendation system at all. 
So when raw_df['recommended'] = True, regret should be 0. If regret =0, 
df.loc[df.converted,value_col].sum() = (~df.converted).sum()*failed_cost, so y = failed_cost
when value_col value =1, X=1, 
failed_cost=raw_df.loc[raw_df.converted,'customer_ltv'].sum()/(~df.converted).sum()

In [90]:
## Use this if you want X to be 1
# cost = raw_df['converted'].sum() / (~raw_df['converted']).sum()

## Use this if you want X to be the customer lifetime value
cost = raw_df['customer_ltv'].sum() / (~raw_df['converted']).sum()


In [91]:
cost

0.07028133092167758

In [131]:
raw_df['recommended'] = True
t, v = split_train_valid(raw_df)
recommendation_regret(raw_df, cost), recommendation_regret(t, cost), recommendation_regret(v, cost)

(1.8189894035458565e-12, -348.0422308683228, 348.04223086832667)

In [93]:
raw_df['recommended'] = False
t, v = split_train_valid(raw_df)
recommendation_regret(raw_df, cost), recommendation_regret(t, cost), recommendation_regret(v, cost)

(-1.8189894035458565e-12, 348.0422308683228, -348.04223086832667)

In [94]:
t.converted.mean(), v.converted.mean()

(0.009269763070041811, 0.008861278420737284)

In [95]:

high_scores = [
    'ENERGY_SUPPLIER_OVO_ENERGY',
    'ENERGY_SUPPLIER_BRITISH_GAS', 
    'ENERGY_SUPPLIER_SCOTTISH_POWER',
    'ENERGY_SUPPLIER_EDF_ENERGY',
    'ENERGY_SUPPLIER_NPOWER', 
    'ENERGY_SUPPLIER_SSE', 
    'ENERGY_SUPPLIER_EON', 
    'ENERGY_SUPPLIER_SHELL_ENERGY', 
]

raw_df['elec_score'] = raw_df.elec_supplier.apply(lambda x: 2 if x in high_scores else 0)
raw_df['elec_score'] += raw_df.elec_supplier == 'ENERGY_SUPPLIER_BULB'

raw_df['gas_score'] = raw_df.gas_supplier.apply(lambda x: 2 if x in high_scores else 0)
raw_df['gas_score'] += raw_df.gas_supplier == 'ENERGY_SUPPLIER_BULB'

In [97]:
raw_df['supply_score'] = raw_df.elec_supplier_start_date.isna().astype(int)
raw_df['supply_score'] += ((raw_df.prospect_creation_date.dt.date - raw_df.elec_supplier_start_date.dt.date).dt.days >= 365)
raw_df['supply_score'] += ((raw_df.prospect_creation_date.dt.date - raw_df.elec_supplier_start_date.dt.date).dt.days >= 365 * 3)


In [98]:
raw_df['switch_score'] = raw_df['elec_score'] + raw_df['gas_score'] + raw_df['supply_score'] 
raw_df['recommended'] = raw_df['switch_score'] >= 4

In [99]:
raw_df.recommended.mean()

0.18973975490417283

In [100]:
raw_df.groupby('recommended')['converted'].mean()

recommended
False    0.009515
True     0.007780
Name: converted, dtype: float64

In [101]:
# On the whole dataset:
# Precision 0.8%
# Recall 16%
# F1 1.4%
truth = raw_df.converted
pred = raw_df.recommended
precision_score(truth, pred), recall_score(truth, pred), f1_score(truth, pred)


(0.007780382563788944, 0.1607017543859649, 0.014842180309806208)

In [102]:
t, v = split_train_valid(raw_df)
recommendation_regret(raw_df, cost), recommendation_regret(t, cost), recommendation_regret(v, cost)


(399.39197647334186, 353.19480149383617, 46.19717497950495)

In [103]:
raw_df['switch_score'].value_counts().sort_index()

0      8234
1    104205
2      8285
3      4966
4     13205
5      3869
6     12359
Name: switch_score, dtype: int64

In [104]:
raw_df.groupby('switch_score')['converted'].mean()

switch_score
0    0.008987
1    0.009606
2    0.007846
3    0.011277
4    0.008633
5    0.009046
6    0.006473
Name: converted, dtype: float64

In [105]:
raw_df.groupby('switch_score')['customer_ltv'].mean()

switch_score
0    9.387324
1    8.105660
2    8.671429
3    7.878182
4    8.485455
5    7.865625
6    9.273611
Name: customer_ltv, dtype: float64

In [106]:
t.recommended.mean(), v.recommended.mean()

(0.1581531779729686, 0.31266752861782976)

In [107]:
for i in range(7):
    raw_df['recommended'] = raw_df['switch_score'] >= i
    cost_at_threhold = recommendation_regret(raw_df, cost)
    print(i, cost_at_threhold)

0 1.8189894035458565e-12
1 186.0086793582244
2 285.7797264765959
3 222.95464612421574
4 399.39197647334186
5 426.0861702819801
6 390.568924774558


In [108]:
def split_by_thresholds(df, column, high_t, mid_t):
    _df = df.groupby(column)['converted'].agg(['mean', 'count']).sort_values('mean').copy()
    high =  _df.loc[_df['mean'] > high_t].index.tolist()
    mid = _df.loc[(_df['mean'] > mid_t) & (_df['mean'] < high_t)].index.tolist()
    return [s for s in high if s != 'ENERGY_SUPPLIER_TELECOM_PLUS'], [s for s in mid if s != 'ENERGY_SUPPLIER_TELECOM_PLUS']


In [109]:
raw_df.groupby('elec_supplier')['converted'].agg(['mean', 'count']).sort_values('mean')


,mean,count
elec_supplier,,
ENERGY_SUPPLIER_OCTOPUS,0.006207,4833
ENERGY_SUPPLIER_EON,0.006579,5776
ENERGY_SUPPLIER_EDF_ENERGY,0.006880,5087
ENERGY_SUPPLIER_TELECOM_PLUS,0.007089,2539
ENERGY_SUPPLIER_BRITISH_GAS,0.008134,9589
ENERGY_SUPPLIER_SHELL_ENERGY,0.008264,2057
ENERGY_SUPPLIER_OTHER,0.008328,10447
ENERGY_SUPPLIER_NPOWER,0.008333,480
ENERGY_SUPPLIER_SCOTTISH_POWER,0.008732,4810


In [111]:
high_scores, mid_scores = split_by_thresholds(raw_df, 'elec_supplier', .009, .008)

raw_df['elec_score'] = raw_df.elec_supplier.apply(lambda x: 2 if x in high_scores else 0)
raw_df['elec_score'] += raw_df.elec_supplier.isin(mid_scores)



In [112]:
raw_df.groupby('gas_supplier')['converted'].agg(['mean', 'count']).sort_values('mean')

,mean,count
gas_supplier,,
ENERGY_SUPPLIER_OCTOPUS,0.003183,3142
ENERGY_SUPPLIER_EON,0.004977,4621
ENERGY_SUPPLIER_NPOWER,0.005556,180
ENERGY_SUPPLIER_BRITISH_GAS,0.006749,7408
ENERGY_SUPPLIER_OTHER,0.006971,4734
ENERGY_SUPPLIER_TELECOM_PLUS,0.006984,1575
ENERGY_SUPPLIER_SCOTTISH_POWER,0.007668,2869
ENERGY_SUPPLIER_EDF_ENERGY,0.008075,3096
ENERGY_SUPPLIER_SHELL_ENERGY,0.008142,1351


In [113]:
high_scores, mid_scores = split_by_thresholds(raw_df, 'gas_supplier', .009, .0065)


raw_df['gas_score'] = raw_df.gas_supplier.apply(lambda x: 2 if x in high_scores else 0)
raw_df['gas_score'] += raw_df.gas_supplier.isin(mid_scores)

In [114]:
def objective(parameters, return_df=False):
    recommendation_threshold, year_threshold, f_y, ya_na = parameters
    raw_df['supply_score'] = raw_df.elec_supplier_start_date.isna().astype(int) * ya_na
    raw_df['supply_score'] += ((raw_df.prospect_creation_date.dt.date - raw_df.elec_supplier_start_date.dt.date).dt.days >= 365 * year_threshold) * f_y
    raw_df['switch_score_test'] = raw_df['elec_score'] + raw_df['gas_score'] + raw_df['supply_score']
    raw_df['recommended'] = raw_df['switch_score_test']  >= recommendation_threshold
    cost_at_threhold = recommendation_regret(raw_df, cost)
    return raw_df if return_df else cost_at_threhold


In [115]:
results = [(p, objective(p)) for p in itertools.product((3, 4, 5), #Scoring threshold
                                                        (0, 1, 2, 3, 4), #Years of supply
                                                        (1, 2), #Score for long supply
                                                        (0, 1))] #Score for missing supply
pd.DataFrame(results).sort_values(1).head(10)


,0,1
7,"(3, 1, 2, 1)",-1962.430012
3,"(3, 0, 2, 1)",-1925.427533
15,"(3, 3, 2, 1)",-1810.402100
5,"(3, 1, 1, 1)",-1789.443535
11,"(3, 2, 2, 1)",-1770.283423
1,"(3, 0, 1, 1)",-1764.377641
19,"(3, 4, 2, 1)",-1678.626963
13,"(3, 3, 1, 1)",-1674.668916
9,"(3, 2, 1, 1)",-1618.977249
17,"(3, 4, 1, 1)",-1583.538483


The (3, 1, 2, 1) ruleset translates into the following rules:

- If the supplier date is 1 year or more in the past, we assign 2 points
- If the supplier date is absent we assign 1 point (as with the current system)
- We set the recommendation threshold at 3 (current system is 4)


In [116]:
df = objective((3, 1, 2, 1), return_df=True).copy()

In [117]:
df.groupby('gas_score')['converted'].mean()

gas_score
0    0.008920
1    0.007472
2    0.024305
Name: converted, dtype: float64

In [118]:
df.groupby('elec_score')['converted'].mean()

elec_score
0    0.008500
1    0.008326
2    0.016668
Name: converted, dtype: float64

In [119]:
df.groupby('supply_score')['converted'].mean()

supply_score
0    0.008319
1    0.009733
2    0.008201
Name: converted, dtype: float64

In [120]:
SPLIT = .8
t, v = split_train_valid(df, SPLIT)
recommendation_regret(raw_df, cost), recommendation_regret(t, cost), recommendation_regret(v, cost)

(-1962.430012101655, -1834.089692774143, -128.34031932751247)

In [121]:
truth = df.converted
pred = df.recommended
precision_score(truth, pred), recall_score(truth, pred), f1_score(truth, pred)


(0.012118067214065505, 0.2785964912280702, 0.023225881939975428)

In [122]:
df.groupby('switch_score_test')['converted'].mean()

switch_score_test
0    0.008304
1    0.008759
2    0.006344
3    0.012615
4    0.009256
5    0.020156
6    0.010115
Name: converted, dtype: float64

In [123]:
df.groupby('recommended')['converted'].mean()

recommended
False    0.008401
True     0.012118
Name: converted, dtype: float64

In [124]:
df.groupby('switch_score_test')['customer_ltv'].mean()

switch_score_test
0    9.011429
1    7.943456
2    9.417308
3    8.546269
4    7.983465
5    9.627368
6    8.277273
Name: customer_ltv, dtype: float64

In [125]:
df.recommended.mean(), t.recommended.mean(), v.recommended.mean()

(0.21119369790424372, 0.18793148154150327, 0.301724953486172)

In [126]:
df.groupby('elec_supplier')['elec_score'].mean().sort_values()

elec_supplier
ENERGY_SUPPLIER_EDF_ENERGY        0
ENERGY_SUPPLIER_EON               0
ENERGY_SUPPLIER_OCTOPUS           0
ENERGY_SUPPLIER_TELECOM_PLUS      0
ENERGY_SUPPLIER_BRITISH_GAS       1
ENERGY_SUPPLIER_NPOWER            1
ENERGY_SUPPLIER_OTHER             1
ENERGY_SUPPLIER_SCOTTISH_POWER    1
ENERGY_SUPPLIER_SHELL_ENERGY      1
ENERGY_SUPPLIER_BULB              2
ENERGY_SUPPLIER_NONE              2
ENERGY_SUPPLIER_OVO_ENERGY        2
ENERGY_SUPPLIER_SSE               2
ENERGY_SUPPLIER_UNKNOWN           2
ENERGY_SUPPLIER_UTILITA           2
Name: elec_score, dtype: int64

In [127]:
df.groupby('gas_supplier')['gas_score'].mean().sort_values()

gas_supplier
ENERGY_SUPPLIER_EON               0
ENERGY_SUPPLIER_NPOWER            0
ENERGY_SUPPLIER_OCTOPUS           0
ENERGY_SUPPLIER_TELECOM_PLUS      0
ENERGY_SUPPLIER_BRITISH_GAS       1
ENERGY_SUPPLIER_EDF_ENERGY        1
ENERGY_SUPPLIER_OTHER             1
ENERGY_SUPPLIER_OVO_ENERGY        1
ENERGY_SUPPLIER_SCOTTISH_POWER    1
ENERGY_SUPPLIER_SHELL_ENERGY      1
ENERGY_SUPPLIER_BULB              2
ENERGY_SUPPLIER_NONE              2
ENERGY_SUPPLIER_UTILITA           2
Name: gas_score, dtype: int64